In [4]:
#RE ROUTING (HACO)
import pandas as pd
import networkx as nx
import random
import math
from collections import defaultdict

# --------------------------
# Hardcoded topology edges
# (Source, Target, Capacity)
# --------------------------
DEFAULT_EDGES = [
   ("Server", "RouterA", 100),
("Server", "RouterB", 200),
("Server", "RouterC", 150),
("RouterA", "Switch1", 140),
("RouterA", "Switch2", 170),
("Switch1", "Switch2", 120),
("Switch2", "user1", 140),
("Switch2", "RouterB", 160),
("RouterB", "Switch3", 80),
("Switch3", "user2", 110),
("Switch3", "Switch2", 100),
("RouterB", "Switch4", 160),
("Switch4", "user3", 120),
("Switch4", "RouterC", 120),
("RouterC", "Switch5", 140),
("Switch5", "Switch6", 180),
("Switch6", "user4", 100),
("Switch6", "Switch4", 200),
("Switch5", "user4", 130),

]

# --------------------------
# Utility: build graph from default edges
# returns undirected graph with capacity on edges
# --------------------------
def build_graph_from_defaults():
    G = nx.Graph()
    for u, v, cap in DEFAULT_EDGES:
        # if multiple parallel entries exist, keep max capacity
        if G.has_edge(u, v):
            # keep max capacity
            G[u][v]['capacity'] = max(G[u][v]['capacity'], float(cap))
        else:
            G.add_edge(u, v, capacity=float(cap))
    return G

# --------------------------
# Edge cost function
# static part: 1 / capacity (lower capacity => higher static cost)
# congestion penalty: if a node in path is RouterA/B/C apply penalty proportional to congestion %
# total edge cost used in ACO for choosing next hop will combine edge static cost and arriving-node congestion
# --------------------------
def build_edge_costs(G, cong_snapshot, node_penalty_weight=1.0):
    # cong_snapshot: dict mapping "RouterA" etc. to congestion %
    # returns a dict edge_costs[(u,v)] = cost
    edge_costs = {}
    for u, v, data in G.edges(data=True):
        cap = float(data.get('capacity', 1.0))
        static_cost = 1.0 / cap  # smaller capacity higher cost
        # congestion penalty: consider congestion at the destination node v if v is RouterA/B/C
        cong_v = cong_snapshot.get(v, 0.0)
        cong_u = cong_snapshot.get(u, 0.0)
        # We'll take avg congestion of endpoints scaled by node_penalty_weight
        node_penalty = node_penalty_weight * ((cong_u + cong_v) / 200.0)  # scale 0..1
        edge_costs[(u, v)] = static_cost + node_penalty
        edge_costs[(v, u)] = static_cost + node_penalty  # undirected
    return edge_costs

# --------------------------
# HACO (simplified ACO) for single source-dest
# returns best path found (list of nodes) and its cost
# --------------------------
def haco_for_pair(G, src, dst, edge_costs, ants=30, iterations=40, alpha=1.0, beta=3.0, evap=0.5, q0=100.0, seed=42):
    random.seed(seed)
    nodes = list(G.nodes())
    # initialize pheromone on edges
    pher = {}
    for u, v in G.edges():
        pher[(u, v)] = 1.0
        pher[(v, u)] = 1.0

    # helper: neighbors of a node
    nbrs = {n: list(G.neighbors(n)) for n in G.nodes()}

    def path_cost(path):
        # sum edge_costs
        c = 0.0
        for i in range(len(path)-1):
            u, v = path[i], path[i+1]
            c += edge_costs.get((u, v), 1e6)
        return c

    # seed pheromone using shortest path (fewest hops) if exists
    try:
        shortest_default = nx.shortest_path(G, source=src, target=dst)
        # deposit a bit of pheromone on edges of default path
        for i in range(len(shortest_default)-1):
            u, v = shortest_default[i], shortest_default[i+1]
            pher[(u, v)] += 5.0
            pher[(v, u)] += 5.0
    except Exception:
        shortest_default = None

    best_path = None
    best_cost = float('inf')
    # ACO loop
    for it in range(iterations):
        all_paths = []
        for a in range(ants):
            # build path probabilistically
            current = src
            visited = {current}
            path = [current]
            # simple greedy limit to avoid infinite loops
            max_hop = len(G.nodes()) * 2
            hop = 0
            while current != dst and hop < max_hop:
                neigh = [n for n in nbrs[current] if n not in visited or n==dst]
                if not neigh:
                    break
                # compute transition probabilities
                probs = []
                denom = 0.0
                for v in neigh:
                    tau = pher[(current, v)] ** alpha
                    eta = (1.0 / edge_costs.get((current, v), 1e6)) ** beta
                    val = tau * eta
                    probs.append((v, val))
                    denom += val
                if denom <= 0:
                    # fall back uniform
                    nxt = random.choice(neigh)
                else:
                    # roulette wheel
                    r = random.random()
                    # normalize
                    cumulative = 0.0
                    picked = None
                    for v, val in probs:
                        cumulative += val/denom
                        if r <= cumulative:
                            picked = v
                            break
                    nxt = picked if picked is not None else probs[-1][0]
                path.append(nxt)
                visited.add(nxt)
                current = nxt
                hop += 1
            # if ended at destination, evaluate
            if path[-1] == dst:
                c = path_cost(path)
                all_paths.append((path, c))
                if c < best_cost:
                    best_cost = c
                    best_path = path
        # global pheromone evaporation
        for e in list(pher.keys()):
            pher[e] *= (1.0 - evap)
            if pher[e] < 1e-6:
                pher[e] = 1e-6
        # deposit pheromone according to path quality
        for path, c in all_paths:
            deposit = q0 / (1.0 + c)
            for i in range(len(path)-1):
                u, v = path[i], path[i+1]
                pher[(u, v)] += deposit
                pher[(v, u)] += deposit
    # return best
    return best_path, best_cost, shortest_default

# --------------------------
# Main driver: dynamic rerouting per timestamp
# --------------------------
def dynamic_reroute_haco(congestion_csv, users=("User1","User2","User3","User4"),
                         node_penalty_weight=5.0,  # larger gives more weight to congestion
                         ants=20, iterations=30,
                         alpha=1.0, beta=2.0, evap=0.5):
    """
    congestion_csv: path to router_congestion_int.csv (columns RouterA,RouterB,RouterC, optional Time)
    For each timestep, prints default path (shortest ignoring congestion) and HACO-updated path (if different).
    """
    df = pd.read_csv(congestion_csv)
    # add Time if missing
    if "Time" not in df.columns:
        df.insert(0, "Time", [i*10 for i in range(len(df))])

    G_default = build_graph_from_defaults()

    for _, row in df.iterrows():
        time_label = row["Time"]
        # build congestion snapshot mapping node->% for RouterA/B/C
        cong_snapshot = {}
        for k in ["RouterA","RouterB","RouterC"]:
            if k in row:
                try:
                    cong_snapshot[k] = float(row[k])
                except:
                    cong_snapshot[k] = 0.0
        print(f"\n=== Time {time_label} sec ===")
        # build edge costs including congestion penalty
        # For HACO we compute costs on a fresh graph each time
        G = build_graph_from_defaults()
        edge_costs = build_edge_costs(G, cong_snapshot, node_penalty_weight=node_penalty_weight)

        # For each user, compute default shortest path (fewest hops) and HACO path
        for user in users:
            src = "Server"
            dst = user
            try:
                default_path = nx.shortest_path(G, source=src, target=dst)  # by hops
            except (nx.NetworkXNoPath, nx.NodeNotFound):
                print(f"{src} → {dst}: no default path found.")
                continue

            # run HACO seeded by default path
            best_path, best_cost, seeded = haco_for_pair(
                G, src, dst, edge_costs,
                ants=ants, iterations=iterations, alpha=alpha, beta=beta, evap=evap,
                seed=hash((time_label, src, dst)) & 0xffffffff
            )

            # If HACO failed (no path), fall back to default
            if best_path is None:
                print(f"{src} → {dst}: default path = {default_path} (HACO found no path).")
                continue

            # Compare
            if best_path == default_path:
                print(f"{src} → {dst}: default path (no change): {default_path}")
            else:
                print(f"{src} → {dst}: BEFORE (default) = {default_path}  ===> AFTER (HACO) = {best_path}")
    print("\nDone.")

# --------------------------
# Example usage:
# --------------------------
dynamic_reroute_haco("C:/Users/Mukta/OneDrive/Desktop/Orange Hackathon/router_congestion_int.csv")

# Adjust HACO params (ants, iterations, node_penalty_weight) to trade speed vs. exploration



=== Time 0 sec ===
Server → User1: no default path found.
Server → User2: no default path found.
Server → User3: no default path found.
Server → User4: no default path found.

=== Time 10 sec ===
Server → User1: no default path found.
Server → User2: no default path found.
Server → User3: no default path found.
Server → User4: no default path found.

=== Time 20 sec ===
Server → User1: no default path found.
Server → User2: no default path found.
Server → User3: no default path found.
Server → User4: no default path found.

=== Time 30 sec ===
Server → User1: no default path found.
Server → User2: no default path found.
Server → User3: no default path found.
Server → User4: no default path found.

=== Time 40 sec ===
Server → User1: no default path found.
Server → User2: no default path found.
Server → User3: no default path found.
Server → User4: no default path found.

=== Time 50 sec ===
Server → User1: no default path found.
Server → User2: no default path found.
Server → User3: n

In [ ]:
# BANDWIDTH ADJUSTMENT
import pandas as pd
import networkx as nx

def adjust_bandwidth(congestion_file, high_thresh=70, low_thresh=40, increase_factor=1.2, timestep=10):
    # Load congestion data
    df_cong = pd.read_csv(congestion_file)

    # Add Time column if missing
    if "Time" not in df_cong.columns:
        df_cong.insert(0, "Time", [i * timestep for i in range(len(df_cong))])

    # Ensure router columns exist
    expected_cols = {"RouterA", "RouterB", "RouterC"}
    if not expected_cols.issubset(set(df_cong.columns)):
        raise ValueError(f"Congestion CSV must contain {expected_cols}, found {df_cong.columns}")

    # --- Hardcoded topology (Source, Target, Capacity) ---
    default_edges = [
        ("User1", "Switch1", 110),
        ("Switch1", "Switch9", 80),
        ("Switch9", "Switch2", 120),
        ("Switch2", "Switch8", 130),
        ("Switch8", "RouterA", 90),
        ("RouterA", "Server", 180),
        ("User1", "Switch8", 140),
        ("Switch8", "Switch9", 180),
        ("Switch9", "Switch8", 190),
        ("Switch8", "Switch10", 180),
        ("Switch8", "Switch4", 140),
        ("Switch10", "User2", 110),
        ("Switch10", "Switch11", 130),
        ("Switch11", "Switch4", 80),
        ("Switch4", "RouterB", 100),
        ("RouterB", "Server", 120),
        ("Switch11", "Switch12", 60),
        ("Switch12", "User3", 160),
        ("Switch12", "Switch14", 100),
        ("Switch14", "RouterC", 130),
        ("RouterC", "Server", 180),
        ("Switch11", "Switch8", 60),
        ("Switch14", "Switch13", 120),
        ("Switch13", "User4", 80),
        ("Switch13", "Switch6", 190),
        ("Switch6", "Switch14", 70),
        ("Switch14", "RouterC", 70),
    ]

    # Iterate over time steps
    for _, row in df_cong.iterrows():
        time_label = row["Time"]
        routers = {col: float(row[col]) for col in df_cong.columns if col.startswith("Router")}

        # Reset graph to default every timestep
        G = nx.Graph()
        for src, tgt, cap in default_edges:
            G.add_edge(src, tgt, capacity=float(cap))

        print(f"\n⏱️ Time step {time_label} sec:")
        for r, load in routers.items():
            if load > high_thresh:
                status = "Congested"
            elif load > low_thresh:
                status = "Moderate"
            else:
                status = "Underloaded"
            print(f"Router {r} at {load:.0f}% → {status}")

        # Track changes
        changes = []
        for r, load in routers.items():
            if load > high_thresh:  # congested router
                neighbors = list(G.neighbors(r))
                underloaded_neighbors = [n for n in neighbors if n in routers and routers[n] < low_thresh]

                if underloaded_neighbors:
                    for n in underloaded_neighbors:
                        old_cap = float(G[r][n]["capacity"])
                        new_cap = old_cap * increase_factor
                        G[r][n]["capacity"] = new_cap
                        changes.append(f"  Bandwidth change: {r}–{n} (old={old_cap:.1f} → new={new_cap:.1f})")
                else:
                    # No underloaded neighbor → adjust busiest edge
                    edges = [(n, float(G[r][n]["capacity"])) for n in neighbors]
                    if edges:
                        n, old_cap = max(edges, key=lambda x: x[1])
                        new_cap = old_cap * increase_factor
                        G[r][n]["capacity"] = new_cap
                        changes.append(f"  QoS adjustment: {r}–{n} (old={old_cap:.1f} → new={new_cap:.1f})")

        if not changes:
            print("  No bandwidth changes at this step.")
        else:
            for c in changes:
                print(c)


# Example usage
congestion_file = "C:/Users/ASUS/Downloads/router_congestion_int.csv"
adjust_bandwidth(congestion_file)



⏱️ Time step 0 sec:
Router RouterA at 1% → Underloaded
Router RouterB at 29% → Underloaded
Router RouterC at 30% → Underloaded
  No bandwidth changes at this step.

⏱️ Time step 10 sec:
Router RouterA at 3% → Underloaded
Router RouterB at 27% → Underloaded
Router RouterC at 68% → Moderate
  No bandwidth changes at this step.

⏱️ Time step 20 sec:
Router RouterA at 35% → Underloaded
Router RouterB at 38% → Underloaded
Router RouterC at 90% → Congested
  QoS adjustment: RouterC–Server (old=180.0 → new=216.0)

⏱️ Time step 30 sec:
Router RouterA at 97% → Congested
Router RouterB at 66% → Moderate
Router RouterC at 3% → Underloaded
  QoS adjustment: RouterA–Server (old=180.0 → new=216.0)

⏱️ Time step 40 sec:
Router RouterA at 45% → Moderate
Router RouterB at 94% → Congested
Router RouterC at 9% → Underloaded
  QoS adjustment: RouterB–Server (old=120.0 → new=144.0)

⏱️ Time step 50 sec:
Router RouterA at 2% → Underloaded
Router RouterB at 16% → Underloaded
Router RouterC at 31% → Underloa

In [ ]:
# POLICING
import pandas as pd
import networkx as nx

def run_policing_only(
    congestion_file,
    high_thresh=70,
    low_thresh=40,
    increase_factor=1.2,
    timestep=10,
):
    """
    Apply only policing (QoS adjustments) when routers are congested and
    no underloaded neighbors exist, across ALL timesteps.
    """

    # Load congestion data
    df_cong = pd.read_csv(congestion_file)

    # Add Time column if missing
    if "Time" not in df_cong.columns:
        df_cong.insert(0, "Time", [i * timestep for i in range(len(df_cong))])

    expected_cols = {"RouterA", "RouterB", "RouterC"}
    if not expected_cols.issubset(set(df_cong.columns)):
        raise ValueError(f"Congestion CSV must contain {expected_cols}, found {df_cong.columns}")

    # Hardcoded default topology
    default_edges = [
        ("User1", "Switch1", 110),
        ("Switch1", "Switch9", 80),
        ("Switch9", "Switch2", 120),
        ("Switch2", "Switch8", 130),
        ("Switch8", "RouterA", 90),
        ("RouterA", "Server", 180),
        ("User1", "Switch8", 140),
        ("Switch8", "Switch9", 180),
        ("Switch9", "Switch8", 190),
        ("Switch8", "Switch10", 180),
        ("Switch8", "Switch4", 140),
        ("Switch10", "User2", 110),
        ("Switch10", "Switch11", 130),
        ("Switch11", "Switch4", 80),
        ("Switch4", "RouterB", 100),
        ("RouterB", "Server", 120),
        ("Switch11", "Switch12", 60),
        ("Switch12", "User3", 160),
        ("Switch12", "Switch14", 100),
        ("Switch14", "RouterC", 130),
        ("RouterC", "Server", 180),
        ("Switch11", "Switch8", 60),
        ("Switch14", "Switch13", 120),
        ("Switch13", "User4", 80),
        ("Switch13", "Switch6", 190),
        ("Switch6", "Switch14", 70),
        ("Switch14", "RouterC", 70),
    ]

    # Iterate over timesteps and print logs
    for _, row in df_cong.iterrows():
        time_label = row["Time"]
        routers = {col: float(row[col]) for col in df_cong.columns if col.startswith("Router")}

        # Reset topology each timestep
        G = nx.Graph()
        for src, tgt, cap in default_edges:
            G.add_edge(src, tgt, capacity=float(cap))

        print(f"\n⏱️ Time step {time_label} sec:")

        # Report router states
        for r, load in routers.items():
            if load > high_thresh:
                status = "Congested"
            elif load > low_thresh:
                status = "Moderate"
            else:
                status = "Underloaded"
            print(f"Router {r} at {load:.0f}% → {status}")

        # Apply policing only
        actions = []
        for r, load in routers.items():
            if load > high_thresh:
                neighbors = list(G.neighbors(r))
                underloaded_neighbors = [n for n in neighbors if n in routers and routers[n] < low_thresh]

                if not underloaded_neighbors and neighbors:
                    busiest = max(neighbors, key=lambda n: G[r][n]["capacity"])
                    old_cap = G[r][busiest]["capacity"]
                    new_cap = old_cap * increase_factor
                    G[r][busiest]["capacity"] = new_cap
                    actions.append(
                        f"QoS adjustment (Policing): {r}–{busiest} (old={old_cap:.1f} → new={new_cap:.1f})"
                    )

        if not actions:
            print("No policing applied this step.")
        else:
            for act in actions:
                print(act)
run_policing_only("C:/Users/ASUS/Downloads/router_congestion_int.csv")



⏱️ Time step 0 sec:
Router RouterA at 1% → Underloaded
Router RouterB at 29% → Underloaded
Router RouterC at 30% → Underloaded
No policing applied this step.

⏱️ Time step 10 sec:
Router RouterA at 3% → Underloaded
Router RouterB at 27% → Underloaded
Router RouterC at 68% → Moderate
No policing applied this step.

⏱️ Time step 20 sec:
Router RouterA at 35% → Underloaded
Router RouterB at 38% → Underloaded
Router RouterC at 90% → Congested
QoS adjustment (Policing): RouterC–Server (old=180.0 → new=216.0)

⏱️ Time step 30 sec:
Router RouterA at 97% → Congested
Router RouterB at 66% → Moderate
Router RouterC at 3% → Underloaded
QoS adjustment (Policing): RouterA–Server (old=180.0 → new=216.0)

⏱️ Time step 40 sec:
Router RouterA at 45% → Moderate
Router RouterB at 94% → Congested
Router RouterC at 9% → Underloaded
QoS adjustment (Policing): RouterB–Server (old=120.0 → new=144.0)

⏱️ Time step 50 sec:
Router RouterA at 2% → Underloaded
Router RouterB at 16% → Underloaded
Router RouterC at

In [ ]:
#ACTIVE QUEUE MANAGEMENT
import pandas as pd

def run_aqm(
    congestion_file,
    high_thresh=80,
    moderate_thresh=50,
    timestep=10,
):
    """
    Apply Active Queue Management (AQM) across all timesteps.
    Simulates early packet drop/marking depending on router load.
    """

    # Load congestion data
    df_cong = pd.read_csv(congestion_file)

    # Add Time column if missing
    if "Time" not in df_cong.columns:
        df_cong.insert(0, "Time", [i * timestep for i in range(len(df_cong))])

    expected_cols = {"RouterA", "RouterB", "RouterC"}
    if not expected_cols.issubset(set(df_cong.columns)):
        raise ValueError(f"Congestion CSV must contain {expected_cols}, found {df_cong.columns}")

    # Iterate over timesteps
    for _, row in df_cong.iterrows():
        time_label = row["Time"]
        routers = {col: float(row[col]) for col in df_cong.columns if col.startswith("Router")}

        print(f"\n⏱️ Time step {time_label} sec:")

        for r, load in routers.items():
            if load >= high_thresh:
                action = f"AQM: Router {r} → Heavy congestion ({load:.0f}%). Aggressive early drop applied."
            elif load >= moderate_thresh:
                action = f"AQM: Router {r} → Moderate load ({load:.0f}%). Random early drop/mark applied."
            else:
                action = f"AQM: Router {r} → Low load ({load:.0f}%). No drops."

            print(action)
run_aqm("C:/Users/ASUS/Downloads/router_congestion_int.csv")



⏱️ Time step 0 sec:
AQM: Router RouterA → Low load (1%). No drops.
AQM: Router RouterB → Low load (29%). No drops.
AQM: Router RouterC → Low load (30%). No drops.

⏱️ Time step 10 sec:
AQM: Router RouterA → Low load (3%). No drops.
AQM: Router RouterB → Low load (27%). No drops.
AQM: Router RouterC → Moderate load (68%). Random early drop/mark applied.

⏱️ Time step 20 sec:
AQM: Router RouterA → Low load (35%). No drops.
AQM: Router RouterB → Low load (38%). No drops.
AQM: Router RouterC → Heavy congestion (90%). Aggressive early drop applied.

⏱️ Time step 30 sec:
AQM: Router RouterA → Heavy congestion (97%). Aggressive early drop applied.
AQM: Router RouterB → Moderate load (66%). Random early drop/mark applied.
AQM: Router RouterC → Low load (3%). No drops.

⏱️ Time step 40 sec:
AQM: Router RouterA → Low load (45%). No drops.
AQM: Router RouterB → Heavy congestion (94%). Aggressive early drop applied.
AQM: Router RouterC → Low load (9%). No drops.

⏱️ Time step 50 sec:
AQM: Router 

In [ ]:
#CONGESTION CONTROL PROTOCOLS (END TO END)
import pandas as pd

def run_end_to_end_cc(
    congestion_file,
    high_thresh=80,
    moderate_thresh=50,
    timestep=10,
    init_rate=1000  # arbitrary units (e.g., Mbps)
):
    """
    Simulate end-to-end congestion control (like TCP AIMD).
    Adjusts sending rate per flow depending on router congestion.
    """

    # Load congestion data
    df_cong = pd.read_csv(congestion_file)

    # Add Time column if missing
    if "Time" not in df_cong.columns:
        df_cong.insert(0, "Time", [i * timestep for i in range(len(df_cong))])

    expected_cols = {"RouterA", "RouterB", "RouterC"}
    if not expected_cols.issubset(set(df_cong.columns)):
        raise ValueError(f"Congestion CSV must contain {expected_cols}, found {df_cong.columns}")

    # Initialize sending rates for 4 users
    rates = {"User1": init_rate, "User2": init_rate, "User3": init_rate, "User4": init_rate}

    # Iterate over timesteps
    for _, row in df_cong.iterrows():
        time_label = row["Time"]
        routers = {col: float(row[col]) for col in df_cong.columns if col.startswith("Router")}

        print(f"\n⏱️ Time step {time_label} sec:")

        # Decision: check worst congestion
        worst_router = max(routers, key=routers.get)
        worst_load = routers[worst_router]

        if worst_load >= high_thresh:
            # Multiplicative Decrease
            for u in rates:
                old = rates[u]
                rates[u] *= 0.5
                print(f"  {u}: High congestion at {worst_router} ({worst_load:.0f}%). "
                      f"Rate reduced {old:.1f} → {rates[u]:.1f}")
        elif worst_load >= moderate_thresh:
            # Keep or slow additive increase
            for u in rates:
                old = rates[u]
                rates[u] += 50
                print(f"  {u}: Moderate load at {worst_router} ({worst_load:.0f}%). "
                      f"Rate adjusted {old:.1f} → {rates[u]:.1f}")
        else:
            # Additive Increase
            for u in rates:
                old = rates[u]
                rates[u] += 100
                print(f"  {u}: Low load at {worst_router} ({worst_load:.0f}%). "
                      f"Rate increased {old:.1f} → {rates[u]:.1f}")
run_end_to_end_cc("C:/Users/ASUS/Downloads/router_congestion_int.csv")



⏱️ Time step 0 sec:
  User1: Low load at RouterC (30%). Rate increased 1000.0 → 1100.0
  User2: Low load at RouterC (30%). Rate increased 1000.0 → 1100.0
  User3: Low load at RouterC (30%). Rate increased 1000.0 → 1100.0
  User4: Low load at RouterC (30%). Rate increased 1000.0 → 1100.0

⏱️ Time step 10 sec:
  User1: Moderate load at RouterC (68%). Rate adjusted 1100.0 → 1150.0
  User2: Moderate load at RouterC (68%). Rate adjusted 1100.0 → 1150.0
  User3: Moderate load at RouterC (68%). Rate adjusted 1100.0 → 1150.0
  User4: Moderate load at RouterC (68%). Rate adjusted 1100.0 → 1150.0

⏱️ Time step 20 sec:
  User1: High congestion at RouterC (90%). Rate reduced 1150.0 → 575.0
  User2: High congestion at RouterC (90%). Rate reduced 1150.0 → 575.0
  User3: High congestion at RouterC (90%). Rate reduced 1150.0 → 575.0
  User4: High congestion at RouterC (90%). Rate reduced 1150.0 → 575.0

⏱️ Time step 30 sec:
  User1: High congestion at RouterA (97%). Rate reduced 575.0 → 287.5
  User2

In [ ]:
#BUFFER MANAGEMENT
import pandas as pd

def run_buffer_management(
    congestion_file,
    high_thresh=80,
    moderate_thresh=50,
    timestep=10,
    buffer_capacity=1000  # per router (in packets, arbitrary units)
):
    """
    Simulate buffer management for each router across timesteps.
    Adjusts buffer occupancy and applies drop policies when overloaded.
    """

    # Load congestion data
    df_cong = pd.read_csv(congestion_file)

    # Add Time column if missing
    if "Time" not in df_cong.columns:
        df_cong.insert(0, "Time", [i * timestep for i in range(len(df_cong))])

    expected_cols = {"RouterA", "RouterB", "RouterC"}
    if not expected_cols.issubset(set(df_cong.columns)):
        raise ValueError(f"Congestion CSV must contain {expected_cols}, found {df_cong.columns}")

    # Initialize buffers for routers
    buffers = {"RouterA": 0, "RouterB": 0, "RouterC": 0}

    # Iterate over timesteps
    for _, row in df_cong.iterrows():
        time_label = row["Time"]
        routers = {col: float(row[col]) for col in df_cong.columns if col.startswith("Router")}

        print(f"\n⏱️ Time step {time_label} sec:")

        for r, load in routers.items():
            # Approximate packets arriving proportional to load
            arrivals = int((load / 100) * (buffer_capacity / 2))  

            # Add arrivals to buffer
            buffers[r] += arrivals

            if load >= high_thresh:
                # Tail drop: buffer capped at capacity
                if buffers[r] > buffer_capacity:
                    dropped = buffers[r] - buffer_capacity
                    buffers[r] = buffer_capacity
                    print(f"  {r}: High congestion ({load:.0f}%). Tail drop {dropped} packets.")
                else:
                    print(f"  {r}: High congestion ({load:.0f}%). Buffer at {buffers[r]}/{buffer_capacity}.")
            elif load >= moderate_thresh:
                # Priority drop: drop 20% of arrivals (simulate dropping low-priority traffic)
                drop = int(arrivals * 0.2)
                buffers[r] = max(0, buffers[r] - drop)
                print(f"  {r}: Moderate load ({load:.0f}%). Priority drop {drop} packets. Buffer={buffers[r]}.")
            else:
                # Low load: buffer drains naturally
                drain = int(buffers[r] * 0.3)
                buffers[r] = max(0, buffers[r] - drain)
                print(f"  {r}: Low load ({load:.0f}%). Buffer drains {drain}. Buffer={buffers[r]}.")
run_buffer_management("C:/Users/ASUS/Downloads/router_congestion_int.csv")



⏱️ Time step 0 sec:
  RouterA: Low load (1%). Buffer drains 1. Buffer=4.
  RouterB: Low load (29%). Buffer drains 43. Buffer=102.
  RouterC: Low load (30%). Buffer drains 45. Buffer=105.

⏱️ Time step 10 sec:
  RouterA: Low load (3%). Buffer drains 5. Buffer=14.
  RouterB: Low load (27%). Buffer drains 71. Buffer=166.
  RouterC: Moderate load (68%). Priority drop 68 packets. Buffer=377.

⏱️ Time step 20 sec:
  RouterA: Low load (35%). Buffer drains 56. Buffer=133.
  RouterB: Low load (38%). Buffer drains 106. Buffer=250.
  RouterC: High congestion (90%). Buffer at 827/1000.

⏱️ Time step 30 sec:
  RouterA: High congestion (97%). Buffer at 618/1000.
  RouterB: Moderate load (66%). Priority drop 66 packets. Buffer=514.
  RouterC: Low load (3%). Buffer drains 252. Buffer=590.

⏱️ Time step 40 sec:
  RouterA: Low load (45%). Buffer drains 252. Buffer=591.
  RouterB: High congestion (94%). Buffer at 984/1000.
  RouterC: Low load (9%). Buffer drains 190. Buffer=445.

⏱️ Time step 50 sec:
  

In [ ]:
#LEAKY BUCKET
import pandas as pd

def run_leaky_bucket(congestion_file, bucket_capacity=200, leak_rate=50, timestep=10):
    """
    Apply Leaky Bucket per router across congestion timesteps.

    Args:
        congestion_file: CSV with Time, RouterA, RouterB, RouterC columns
        bucket_capacity: max buffer size (arbitrary units)
        leak_rate: fixed leak rate per timestep
        timestep: interval in seconds between samples
    """
    df = pd.read_csv(congestion_file)

    # Ensure Time column exists
    if "Time" not in df.columns:
        df.insert(0, "Time", [i * timestep for i in range(len(df))])

    routers = [c for c in df.columns if c.startswith("Router")]
    buckets = {r: 0 for r in routers}  # initialize bucket level per router

    for _, row in df.iterrows():
        time_label = row["Time"]
        print(f"\n⏱️ Time step {time_label} sec:")

        for r in routers:
            load = float(row[r])  # congestion percentage

            # Convert congestion % to incoming traffic (scaled)
            arrivals = int((load / 100) * bucket_capacity * 0.5)

            # Leak traffic
            leaked = min(buckets[r], leak_rate)
            buckets[r] -= leaked

            # Attempt to add arrivals
            if buckets[r] + arrivals <= bucket_capacity:
                buckets[r] += arrivals
                action = "queued"
            else:
                dropped = (buckets[r] + arrivals) - bucket_capacity
                buckets[r] = bucket_capacity
                action = f"dropped {dropped}"

            print(f"  {r}: load={load:.0f}%, arrivals={arrivals}, leaked={leaked}, "
                  f"bucket={buckets[r]}/{bucket_capacity}, action={action}")
congestion_file = "C:/Users/ASUS/Downloads/router_congestion_int.csv"
run_leaky_bucket(congestion_file)



⏱️ Time step 0 sec:
  RouterA: load=1%, arrivals=1, leaked=0, bucket=1/200, action=queued
  RouterB: load=29%, arrivals=28, leaked=0, bucket=28/200, action=queued
  RouterC: load=30%, arrivals=30, leaked=0, bucket=30/200, action=queued

⏱️ Time step 10 sec:
  RouterA: load=3%, arrivals=3, leaked=1, bucket=3/200, action=queued
  RouterB: load=27%, arrivals=27, leaked=28, bucket=27/200, action=queued
  RouterC: load=68%, arrivals=68, leaked=30, bucket=68/200, action=queued

⏱️ Time step 20 sec:
  RouterA: load=35%, arrivals=35, leaked=3, bucket=35/200, action=queued
  RouterB: load=38%, arrivals=38, leaked=27, bucket=38/200, action=queued
  RouterC: load=90%, arrivals=90, leaked=50, bucket=108/200, action=queued

⏱️ Time step 30 sec:
  RouterA: load=97%, arrivals=97, leaked=35, bucket=97/200, action=queued
  RouterB: load=66%, arrivals=66, leaked=38, bucket=66/200, action=queued
  RouterC: load=3%, arrivals=3, leaked=50, bucket=61/200, action=queued

⏱️ Time step 40 sec:
  RouterA: load

In [ ]:
#TOKEN BUCKET
import pandas as pd

def run_token_bucket(congestion_file, bucket_capacity=200, token_rate=50, timestep=10):
    """
    Apply Token Bucket per router across congestion timesteps.

    Args:
        congestion_file: CSV with Time, RouterA, RouterB, RouterC columns
        bucket_capacity: max number of tokens
        token_rate: tokens added per timestep
        timestep: interval in seconds between samples
    """
    df = pd.read_csv(congestion_file)

    # Ensure Time column exists
    if "Time" not in df.columns:
        df.insert(0, "Time", [i * timestep for i in range(len(df))])

    routers = [c for c in df.columns if c.startswith("Router")]
    tokens = {r: bucket_capacity for r in routers}  # start full

    for _, row in df.iterrows():
        time_label = row["Time"]
        print(f"\n⏱️ Time step {time_label} sec:")

        for r in routers:
            load = float(row[r])  # congestion percentage

            # Convert congestion % to traffic demand
            demand = int((load / 100) * bucket_capacity * 0.5)

            # Refill tokens
            tokens[r] = min(bucket_capacity, tokens[r] + token_rate)

            # Serve traffic
            if demand <= tokens[r]:
                tokens[r] -= demand
                action = "sent"
            else:
                dropped = demand - tokens[r]
                tokens[r] = 0
                action = f"dropped {dropped}"

            print(f"  {r}: load={load:.0f}%, demand={demand}, "
                  f"tokens left={tokens[r]}/{bucket_capacity}, action={action}")
congestion_file = "C:/Users/ASUS/Downloads/router_congestion_int.csv"
run_token_bucket(congestion_file)



⏱️ Time step 0 sec:
  RouterA: load=1%, demand=1, tokens left=199/200, action=sent
  RouterB: load=29%, demand=28, tokens left=172/200, action=sent
  RouterC: load=30%, demand=30, tokens left=170/200, action=sent

⏱️ Time step 10 sec:
  RouterA: load=3%, demand=3, tokens left=197/200, action=sent
  RouterB: load=27%, demand=27, tokens left=173/200, action=sent
  RouterC: load=68%, demand=68, tokens left=132/200, action=sent

⏱️ Time step 20 sec:
  RouterA: load=35%, demand=35, tokens left=165/200, action=sent
  RouterB: load=38%, demand=38, tokens left=162/200, action=sent
  RouterC: load=90%, demand=90, tokens left=92/200, action=sent

⏱️ Time step 30 sec:
  RouterA: load=97%, demand=97, tokens left=103/200, action=sent
  RouterB: load=66%, demand=66, tokens left=134/200, action=sent
  RouterC: load=3%, demand=3, tokens left=139/200, action=sent

⏱️ Time step 40 sec:
  RouterA: load=45%, demand=45, tokens left=108/200, action=sent
  RouterB: load=94%, demand=94, tokens left=90/200, a

In [ ]:
#CHOOSING MULTIPLE BEST
import pandas as pd

METHODS = {
    1: 'Dynamic Rerouting (Hybrid Ant Colony Optimization)',
    2: 'Bandwidth Adjustment and QoS',
    3: 'Traffic Shaping and Policing',
    4: 'Active Queue Management (AQM)',
    5: 'Congestion Control Protocols (End-to-End)',
    6: 'Buffer Management',
    7: 'Leaky Bucket',
    8: 'Token Bucket'
}

def pick_methods(a, b, c):
    """Return list of method IDs to apply based on congestion states."""
    methods = []

    # High congestion > 70%
    if a > 70 or b > 70 or c > 70:
        methods.append(1)   # Dynamic rerouting
        methods.append(8)   # Token bucket shaping
        methods.append(4)   # AQM
        methods.append(5)   # End-to-end congestion control
        methods.append(6)   # Buffer management

    # Moderate congestion 40–70%
    if 40 <= a <= 70 or 40 <= b <= 70 or 40 <= c <= 70:
        methods.append(2)   # Bandwidth Adjustment
        methods.append(3)   # Traffic shaping & policing
        methods.append(7)   # Leaky bucket

    # Low congestion < 40%
    if a < 40 and b < 40 and c < 40:
        methods.append(5)   # Normal TCP congestion control only

    return sorted(set(methods))  # remove duplicates


def assign_methods(congestion_csv, timestep=10):
    df = pd.read_csv(congestion_csv)

    if "Time" not in df.columns:
        df.insert(0, "Time", [i*timestep for i in range(len(df))])

    results = []
    for _, row in df.iterrows():
        t = row["Time"]
        A, B, C = row["RouterA"], row["RouterB"], row["RouterC"]

        method_ids = pick_methods(A, B, C)
        method_names = [METHODS[mid] for mid in method_ids]

        results.append({
            "Time": t,
            "RouterA": A,
            "RouterB": B,
            "RouterC": C,
            "ChosenMethodsIDs": method_ids,
            "ChosenMethods": method_names
        })

    return pd.DataFrame(results)


# Example usage:
# Example usage:
df_out = assign_methods("C:/Users/ASUS/Downloads/router_congestion_int.csv")
print(df_out.to_string(index=False))  # <-- prints ALL rows with all timestamps



 Time  RouterA  RouterB  RouterC         ChosenMethodsIDs                                                                                                                                                                                                                             ChosenMethods
    0        1       29       30                      [5]                                                                                                                                                                                               [Congestion Control Protocols (End-to-End)]
   10        3       27       68                [2, 3, 7]                                                                                                                                                                [Bandwidth Adjustment and QoS, Traffic Shaping and Policing, Leaky Bucket]
   20       35       38       90          [1, 4, 5, 6, 8]                                                                   

In [ ]:
# CHOOSING BEST SINGLE
import pandas as pd

METHODS = {
    1: 'Dynamic Rerouting (Hybrid Ant Colony Optimization)',
    2: 'Bandwidth Adjustment and QoS',
    3: 'Traffic Shaping and Policing',
    4: 'Active Queue Management (AQM)',
    5: 'Congestion Control Protocols (End-to-End)',
    6: 'Buffer Management',
    7: 'Leaky Bucket',
    8: 'Token Bucket'
}

def pick_best_method(a, b, c):
    """
    Return a single best method ID based on refined congestion rules.
    Uses maximum router load as the decision driver.
    """
    max_load = max(a, b, c)

    if max_load > 85:
        return 1   # Dynamic Rerouting
    elif max_load > 70:
        return 4   # AQM
    elif max_load > 60:
        return 3   # Traffic Shaping / Policing
    elif max_load > 50:
        return 2   # Bandwidth Adjustment
    elif max_load > 40:
        return 8   # Token Bucket
    elif max_load > 30:
        return 7   # Leaky Bucket
    elif max_load > 20:
        return 6   # Buffer Management
    else:
        return 5   # End-to-End Congestion Control

def assign_best_methods(congestion_csv, timestep=10):
    """
    Assigns a single 'best' congestion control method per time step
    based on router congestion percentages.
    """
    df = pd.read_csv(congestion_csv)

    # Add synthetic time column if missing
    if "Time" not in df.columns:
        df.insert(0, "Time", [i*timestep for i in range(len(df))])

    expected_cols = {"RouterA", "RouterB", "RouterC"}
    if not expected_cols.issubset(set(df.columns)):
        raise ValueError(f"Congestion CSV must contain {expected_cols}, found {df.columns}")

    results = []
    for _, row in df.iterrows():
        t = row["Time"]
        A, B, C = row["RouterA"], row["RouterB"], row["RouterC"]

        best_id = pick_best_method(A, B, C)
        best_name = METHODS[best_id]

        results.append({
            "Time": t,
            "RouterA": A,
            "RouterB": B,
            "RouterC": C,
            "ChosenMethodID": best_id,
            "ChosenMethod": best_name
        })

    return pd.DataFrame(results)


# === Example usage ===
congestion_file = "C:/Users/ASUS/Downloads/router_congestion_int.csv"

df_out = assign_best_methods(congestion_file)

# Print all timestamps in a clean format
print(df_out.to_string(index=False))

# Optionally save to CSV for easy viewing
df_out.to_csv("C:/Users/ASUS/Downloads/best_methods_output.csv", index=False)


 Time  RouterA  RouterB  RouterC  ChosenMethodID                                       ChosenMethod
    0        1       29       30               6                                  Buffer Management
   10        3       27       68               3                       Traffic Shaping and Policing
   20       35       38       90               1 Dynamic Rerouting (Hybrid Ant Colony Optimization)
   30       97       66        3               1 Dynamic Rerouting (Hybrid Ant Colony Optimization)
   40       45       94        9               1 Dynamic Rerouting (Hybrid Ant Colony Optimization)
   50        2       16       31               7                                       Leaky Bucket
   60       36        9       62               3                       Traffic Shaping and Policing
   70       23       34       29               7                                       Leaky Bucket
   80       22       95       83               1 Dynamic Rerouting (Hybrid Ant Colony Optimization)
